In [ ]:
from music21 import *
import glob

In [ ]:
%pwd

In [ ]:
PATH = "ml/Untitled Folder/music_rnn/bach"

files = glob.glob(PATH + '/**/*.mid', recursive=True)

for file in files:
    try:
        mid = MidiFile(file)
        find_base(mid)
        for i, track in enumerate(mid.tracks):
            print('{} : Track {}: {}'.format(file, i, track.name))
            _, global_min_note, global_max_note = track_to_list(track, base, global_min_note, global_max_note)
    except:
        print("Skipping: ", file)

In [ ]:
s = converter.parse('~/ml/Untitled Folder/music_rnn/bach_clavier/gold/aria.mid')

In [ ]:
k = s.analyze('key')
        #if "major" in k: print("MAJOR")
        #i = interval.Interval(k.tonic, pitch.Pitch('C'))
        #print(k, i)
        #sNew = s.transpose(i)
        #kNew = sNew.analyze("key")
        #print("New key: ", kNew)

In [ ]:
if "major" in k.name: print("minor")

In [ ]:
l, global_min_note, global_max_note, tracks, maxk = track_to_list(s, global_min_note, global_max_note)

In [ ]:
l

In [1]:
from music21 import *
import glob
import numpy as np
import pandas as pd
from os.path import basename
from os.path import isfile
from pathlib import Path

global_min_note, global_max_note = 127,0
MAX_TRACK = 16

def track_to_list(s, min_note, max_note):
    ind = 0
    maxk = 0
    my_index = range(50000)
    my_columns = ["Voice 0", "Voice 1", "Voice 2", "Voice 3", "Voice 4", "Voice 5","Voice 6","Voice 7","Voice 8","Voice 9","Voice 10","Voice 11","Voice 12","Voice 13", "Voice 14", "Voice 15"]
    table = pd.DataFrame(" ", index = my_index, columns = my_columns)
    for p in s.parts:
        column = "Voice {}".format(ind)
        print(column)
        if ind<MAX_TRACK: 
            for n in p.notes:
                if n.isNote: 
                    #print(n)
                    offset = int(n.offset*4//1)
                    duration = int(n.duration.quarterLength*4//1)
                    note = n.midi+33
                    if note<min_note: min_note = note
                    if note>max_note: max_note = note
                    #print(ind, offset, duration, n.midi)
                    for i in range(offset, offset+duration):
                        line = chr(note)
                        table[column][i] = line
                        #print("table['{}'][{}] = {}".format(column, i, line))
                        if i>maxk: maxk = offset
                    #print(ind, k, table[ind, k])
            ind += 1
    #table.fillna(" ", inplace = True)
    return table, min_note, max_note, ind-1, maxk
        
def write_to_file(l, filename, length, voices):
    file = open(filename, "w")
    for r in range(length):
        lst = []
        for c in range(voices+1):
            column = "Voice {}".format(c)
            lst.append(l[column][r])
            line = ''.join(lst)+"!"
        file.write(line)
    file.write("********")
    file.close()
    
def chunks(s, n):
    """Produce `n`-character chunks from `s`."""
    for start in range(0, len(s), n):
        yield s[start:start+n]
    
def read_from_file(filename):
    my_index = range(50000)
    my_columns = ["Voice 0", "Voice 1", "Voice 2", "Voice 3", "Voice 4", "Voice 5","Voice 6","Voice 7","Voice 8","Voice 9","Voice 10","Voice 11","Voice 12","Voice 13", "Voice 14", "Voice 15"]
    table = pd.DataFrame(" ", index = my_index, columns = my_columns)
    with open(filename) as file:
        text = file.read()
        #print(text)
    file.close()
    words = "!".split(text)
    ind = 0
    for chunk in chunks(text, 4):
        #print(chunk)
        for i in range(4):
            table[i, ind]=ord(chunk[i])
        ind += 1
    return table, ind
        
    
def list_to_stream(table, maxk):
    s = stream.Score()
    for i in range(4):
        if (table[i].sum() - 320000) > 0:
            print("Voice:", i)
            p = stream.Part()
            ind = 0
            k = 0
            duration = 1
            while k < maxk:
                midi_pitch = table[i,k]-32
                if midi_pitch > 0:
                    next_midi_pitch = table[i, k+1]
                    if midi_pitch == next_midi_pitch:
                        duration += 1
                    else:
                        n = note.Note()
                        n.duration.quarterLength = duration / 4
                        n.pitch.midi = midi_pitch
                        p.append(n)
                        duration = 1
                k += 1
            s.insert(0,p)
            print("appending", p)
    return s


In [2]:
PATH = "../../ml/Untitled Folder/music_rnn/bach_new"

files = glob.glob(PATH + '/**/*.mid', recursive=True)
ind=0
bad_files = []
for file in files:
    logfilename = Path(PATH + "/txt/log/"+basename(file).split(".")[0]+".log")
    if not logfilename.is_file():
        try:
            print("Trying: ", logfilename)
            s = converter.parse(file)
            k = s.analyze('key')
            i = interval.Interval(k.tonic, pitch.Pitch('C'))
            print(k, i)
            sNew = s.transpose(i)
            kNew = sNew.analyze("key")
            print("New key: ", kNew)
        except:
            print("Skipping: ", file)
            bad_files.append(file)
        if "major" in k.name: scale = "major"
        else: scale = "minor"
        l, global_min_note, global_max_note, tracks, maxk = track_to_list(sNew, global_min_note, global_max_note)
        print(ind, file, tracks, global_min_note, global_max_note, maxk)
        txtfilename = PATH + "/txt/"+scale+"/"+str(tracks+1)+"/"+basename(file).split(".")[0]+".txt"
        print(ind, txtfilename)
        write_to_file(l, txtfilename, maxk, tracks)
        logfile = open(logfilename, "w")
        logtext = "#{}:{} {}, {} voices, len: {}".format(ind, txtfilename, scale, tracks, maxk)
        logfile.write(logtext)
        logfile.close()
        ind += 1
    else: print(logfilename, " exists. skipping")
    
print("Global max: {}, global min: {}, alphabet_size: {}"
      .format(global_min_note, global_max_note, global_max_note-global_min_note+1+1))

Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv1041b.log
C major <music21.interval.Interval P1>
New key:  C major
Voice 0


/Users/Zufek/ml/env/lib/python3.6/site-packages/ipykernel_launcher.py:27: Music21DeprecationWarning: Note._getMidi was deprecated on May 2014 and will disappear at or after September 2017, v.5. use pitch.midi instead


Voice 1
Voice 2
Voice 3
Voice 4
Voice 5
Voice 6
0 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1041b.mid 6 57 124 728
0 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/7/bwv1041b.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/1080-c12.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
1 ../../ml/Untitled Folder/music_rnn/bach_new/1080-c12.mid 1 57 124 1602
1 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/2/1080-c12.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/1079-02.log
c minor <music21.interval.Interval P1>
New key:  c minor
Voice 0
Voice 1
Voice 2
2 ../../ml/Untitled Folder/music_rnn/bach_new/1079-02.mid 2 57 124 240
2 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/3/1079-02.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv1077.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
Voice 4
3 ../../ml/Untitled Folder/music_rnn/bach_new/bwv10

New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
27 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1073.mid 3 48 124 448
27 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/4/bwv1073.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv525-3.log
E- major <music21.interval.Interval m-3>
New key:  C major
Voice 0
Voice 1
Voice 2
28 ../../ml/Untitled Folder/music_rnn/bach_new/bwv525-3.mid 2 48 124 1524
28 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/3/bwv525-3.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/1080-c02.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
29 ../../ml/Untitled Folder/music_rnn/bach_new/1080-c02.mid 3 48 124 1336
29 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/4/1080-c02.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv802.log
a minor <music21.interval.Interval M-6>
New key:  c minor
Voice 0
Voice 1
30 ../../ml/Untitled Folder/music_rnn/bach_new/b

New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
53 ../../ml/Untitled Folder/music_rnn/bach_new/bwv653-2.mid 3 40 124 984
53 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/4/bwv653-2.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv552p.log
E- major <music21.interval.Interval m-3>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
54 ../../ml/Untitled Folder/music_rnn/bach_new/bwv552p.mid 3 40 124 3264
54 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/4/bwv552p.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv0248f.log
D major <music21.interval.Interval M-2>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
Voice 4
Voice 5
Voice 6
Voice 7
Voice 8
Voice 9
Voice 10
Voice 11
Voice 12
Voice 13
Voice 14
Voice 15
Voice 16
Voice 16
55 ../../ml/Untitled Folder/music_rnn/bach_new/bwv0248f.mid 15 40 124 6536
55 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/16/bwv0248f.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv5

...

New key:  c minor
Voice 0
Voice 1
Voice 2
278 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1014/sonat_1b.mid 2 40 129 2240
278 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/3/sonat_1b.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/sonat_1c.log
D major <music21.interval.Interval M-2>
New key:  C major
Voice 0
Voice 1
Voice 2
279 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1014/sonat_1c.mid 2 40 129 456
279 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/3/sonat_1c.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/sonat_1d.log
b minor <music21.interval.Interval M-7>
New key:  c minor
Voice 0
Voice 1
Voice 2
280 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1014/sonat_1d.mid 2 40 129 1452
280 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/3/sonat_1d.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/988-v09.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
Voice 2
281 ../../ml/Untitled Fo

Voice 1
Voice 2
304 ../../ml/Untitled Folder/music_rnn/bach_new/bwv988x/988-v12.mid 2 40 129 764
304 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/3/988-v12.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/988-v13.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
Voice 2
305 ../../ml/Untitled Folder/music_rnn/bach_new/bwv988x/988-v13.mid 2 40 129 764
305 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/3/988-v13.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/988-v07.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
306 ../../ml/Untitled Folder/music_rnn/bach_new/bwv988x/988-v07.mid 1 40 129 762
306 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/2/988-v07.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/988-v11.log
e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
Voice 1
307 ../../ml/Untitled Folder/music_rnn/bach_new/bwv988x/988-v11.mid 1 40 129

F major <music21.interval.Interval P-4>
New key:  C major
Voice 0
333 ../../ml/Untitled Folder/music_rnn/bach_new/846-869/bwv856.mid 0 40 129 0
333 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/1/bwv856.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv857.log
f minor <music21.interval.Interval P-4>
New key:  c minor
Voice 0
334 ../../ml/Untitled Folder/music_rnn/bach_new/846-869/bwv857.mid 0 40 129 0
334 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/1/bwv857.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv869.log
b minor <music21.interval.Interval M-7>
New key:  c minor
Voice 0
335 ../../ml/Untitled Folder/music_rnn/bach_new/846-869/bwv869.mid 0 40 129 0
335 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/1/bwv869.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/bwv855.log
e minor <music21.interval.Interval M-3>
New key:  c minor
Voice 0
336 ../../ml/Untitled Folder/music_rnn/bach_new/846-869/bwv855.mid 0 40 1

New key:  c minor
Voice 0
Voice 1
Voice 2
Voice 3
360 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1009/cs3-3cou.mid 3 40 129 2016
360 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/4/cs3-3cou.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/cs3-1pre.log
C major <music21.interval.Interval P1>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
361 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1009/cs3-1pre.mid 3 40 129 1044
361 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/4/cs3-1pre.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/cs3-6gig.log
G major <music21.interval.Interval P-5>
New key:  C major
Voice 0
Voice 1
Voice 2
Voice 3
362 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1009/cs3-6gig.mid 3 40 129 1296
362 ../../ml/Untitled Folder/music_rnn/bach_new/txt/major/4/cs3-6gig.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/cs3-4sar.log
C major <music21.interval.Interval P1>
New key:  C major
Voice 0
Voice 1
Voice 2


g minor <music21.interval.Interval P-5>
New key:  c minor
Voice 0
Voice 1
Voice 2
387 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1001/vs1-4prs.mid 2 40 129 1626
387 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/3/vs1-4prs.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/cs2-5men.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
Voice 2
388 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1008/cs2-5men.mid 2 40 129 1428
388 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/3/cs2-5men.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/cs2-3cou.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0
Voice 1
Voice 2
389 ../../ml/Untitled Folder/music_rnn/bach_new/bwv1008/cs2-3cou.mid 2 40 129 776
389 ../../ml/Untitled Folder/music_rnn/bach_new/txt/minor/3/cs2-3cou.txt
Trying:  ../../ml/Untitled Folder/music_rnn/bach_new/txt/log/cs2-1pre.log
d minor <music21.interval.Interval M-2>
New key:  c minor
Voice 0

In [ ]:
tracks

###### from os.path import basename

global_min_note, global_max_note = 127,0
s = converter.parse('~/ml/Untitled Folder/music_rnn/bach_clavier/partitas/all2.mid')
filename = basename('~/ml/Untitled Folder/music_rnn/bach_clavier/partitas/all2.mid').split(".")[0]+".txt"
print(filename)
l, global_min_note, global_max_note, i, maxk = track_to_list(s, global_min_note, global_max_note)

In [ ]:
127-12

In [ ]:
s.parts

In [ ]:
files

In [ ]:
import jupyter_core

In [ ]:
jrd = jupyter_core.paths.jupyter_runtime_dir()

In [ ]:
jrd